<a href="https://colab.research.google.com/github/marco-siino/llm-network-testbed/blob/main/knowledge-extraction/Q1/Codestral/Codestral_LogKnowledgeExtraction_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries and load the model.

In [50]:
import os
import random
import time
import requests
!pip install rouge-score

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

!pip install mistralai
from mistralai import Mistral

#api_key = os.environ['7yOu0lH5XcZC1U2ZUI2uv0ghBDduhHp5']

client = Mistral(api_key='7yOu0lH5XcZC1U2ZUI2uv0ghBDduhHp5')

model = "codestral-mamba-latest"

message = [
    {
        "role": "user",
        "content": "Write a function for fibonacci"
    }
]

chat_response = client.chat.complete(
    model=model,
    messages=message
)
print(chat_response.choices[0].message.content)

Sure! Here's an example of a Python function that generates a Fibonacci sequence:

```python
def fibonacci(n):
    # Create an empty list to store the Fibonacci sequence
    fib_sequence = []

    # The first two numbers in the Fibonacci sequence are 0 and 1
    if n >= 1:
        fib_sequence.append(0)
    if n >= 2:
        fib_sequence.append(1)

    # Generate the remaining numbers in the sequence
    for i in range(2, n):
        next_number = fib_sequence[i-1] + fib_sequence[i-2]
        fib_sequence.append(next_number)

    return fib_sequence
```

You can use this function to generate the Fibonacci sequence up to a given number. For example, `fibonacci(10)` will return `[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]`.


In [4]:
!nvidia-smi

Mon Feb 24 08:58:34 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Download and organize the dataset.

In [6]:
url = "https://github.com/marco-siino/llm-network-testbed/raw/refs/heads/main/knowledge-extraction/Q1/prompt_Q1-2024-05-20.zip"
output_file = "prompt_Q1-2024-05-20.zip"

os.system(f"wget -O {output_file} {url}")

0

In [7]:
!unzip prompt_Q1-2024-05-20.zip

Archive:  prompt_Q1-2024-05-20.zip
  inflating: prompt_Q1-2024-05-20/prompt_n_hours=2_skip_hours=1060.txt  
  inflating: prompt_Q1-2024-05-20/prompt_n_hours=2_skip_hours=1072.txt  
  inflating: prompt_Q1-2024-05-20/prompt_n_hours=2_skip_hours=1090.txt  
  inflating: prompt_Q1-2024-05-20/prompt_n_hours=2_skip_hours=1004.txt  
  inflating: prompt_Q1-2024-05-20/prompt_n_hours=2_skip_hours=1020.txt  
  inflating: prompt_Q1-2024-05-20/prompt_n_hours=2_skip_hours=990.txt  
  inflating: prompt_Q1-2024-05-20/prompt_n_hours=2_skip_hours=1068.txt  
  inflating: prompt_Q1-2024-05-20/prompt_n_hours=2_skip_hours=1076.txt  
  inflating: prompt_Q1-2024-05-20/prompt_n_hours=2_skip_hours=1078.txt  
  inflating: prompt_Q1-2024-05-20/prompt_n_hours=2_skip_hours=1010.txt  
  inflating: prompt_Q1-2024-05-20/prompt_n_hours=2_skip_hours=1084.txt  
  inflating: prompt_Q1-2024-05-20/prompt_n_hours=2_skip_hours=1006.txt  
  inflating: prompt_Q1-2024-05-20/prompt_n_hours=2_skip_hours=1070.txt  
  inflating: prom

In [8]:
folder_path="prompt_Q1-2024-05-20"
# List all files in the folder
files = os.listdir(folder_path)

# Sort the files alphabetically
files.sort()

# Function to pre-process the few shots.

In [9]:
# The following function add the Mistral Tags within the file content.
def preprocess_sample(file_contents):

                sample ="<s>[INST]Given the following LOG, how many frames are received per each dev_eui?\n\nLOG:\n"
                modified_string = file_contents.replace("QUESTION: How many packets are received for each device? ", "")
                modified_string = modified_string.replace("\n\nANSWER:", "ANSWER:[/INST]")
                modified_string = modified_string[:-1]

                sample += modified_string+"</s>"
                sample += "\n\n"

                return sample


# Create the few-shots samples.

In [10]:
def create_few_shots_set(folder_path,nr_samples):

    # Counter to keep track of the number of files read
    count = 0

    few_shots_set = ""

    # Get a list of all files in the directory
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

    # Shuffle the list of files. Seed set - equal to 4 - for reproducibility.
    random.Random(4).shuffle(files)

    global test_set
    test_set = files

    # Iterate through all files in the folder
    for filename in files:
        print(filename)
        # Check if the file is a text file
        if filename.endswith(".txt"):
            # Open the file and read its contents
            with open(os.path.join(folder_path, filename), 'r') as file:
                # Read the contents of the file and append it to all_contents
                file_contents = file.read()

                few_shots_set+=preprocess_sample(file_contents)

                # Increment the counter
                count += 1

                # Check if reached the limit of n files
                if count == n:
                    break
        test_set.remove(filename)

    return few_shots_set

In [11]:
# Specify the number of files to read
n = 5  # Example: Read the first 5 files

# Call the function to read text files in the folder and concatenate their contents
few_shots_samples = create_few_shots_set(folder_path, n)

# Print or use the concatenated contents as needed
print(few_shots_samples)

prompt_n_hours=2_skip_hours=276.txt
prompt_n_hours=2_skip_hours=492.txt
prompt_n_hours=2_skip_hours=502.txt
prompt_n_hours=2_skip_hours=482.txt
prompt_n_hours=2_skip_hours=702.txt
<s>[INST]Given the following LOG, how many frames are received per each dev_eui?

LOG:
time: 2019-05-31 12:22:31.397000, freq: 868.3, chan: 22, lsnr: -3.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 4130
time: 2019-05-31 12:22:37.593000, freq: 868.5, chan: 23, lsnr: -1.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 4130
time: 2019-05-31 12:43:18.611000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -85.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 4332
time: 2019-05-31 12:45:06.829000, freq: 868.1, chan: 21, lsnr: 15.0, rssi: -89.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 430
time: 2019-05-31 13:10:39.845000, freq: 868.3, chan: 22, lsnr: -4.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 25, FCnt: 4132
t

# Chat with the model to get the results.

In [53]:
# Define the directory path containing the results.
result_folder_name = 'codestral_results'
result_directory_path = os.path.join(os.getcwd(), result_folder_name)
dataset_path = 'prompt_Q1-2024-05-20'

# Iterate through all the test_files
for filename in test_set:
        print(filename)
        if(os.path.isfile(result_directory_path+"/"+filename)):
          print("Already done!\n\n")
          continue
        # Check if the file is a text file
        if filename.endswith(".txt"):

            # Create the directory if it doesn't exist
            if not os.path.exists(result_directory_path):
                os.makedirs(result_directory_path)

            # Open the file and read its contents
            with open(os.path.join(dataset_path, filename), 'r') as file:
                # Read the contents of the file and append it to all_contents
                file_contents = file.read()

                prompt = few_shots_samples + "\n[INST]Given the following LOG, how many frames are received per each dev_eui?\n"

                # Split the string at the occurrence of "ANSWER:"
                parts = file_contents.split("ANSWER:", 1)

                parts[0] = parts[0].replace("\n\nQUESTION: How many packets are received for each device? ", "")

                # Take only until ANSWER
                prompt+= parts[0]+"ANSWER: [/INST]"

                print(prompt)

                message = [
                    {
                        "role": "user",
                        "content": prompt
                    }
                ]

                success = False
                while not success:
                    try:
                        chat_response = client.chat.complete(
                            model=model,
                            messages=message
                        )
                        success = True  # Se non viene sollevata alcuna eccezione, imposta success su True
                    except Exception as e:
                        print(f"Errore: {e}. Riprovo tra 20 secondi...")
                        time.sleep(20)  # Attendi 20 secondi prima di ritentare


                result = chat_response.choices[0].message.content
                print(result+"\n\n")

                # Define the file path
                file_path = os.path.join(result_directory_path, filename)

                with open(file_path, 'w') as f:
                  # Write some content to the file
                  f.write(result)

Streaming output truncated to the last 5000 lines.
time: 2019-06-09 22:35:03.260000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 4437
time: 2019-06-09 22:40:03.644000, freq: 868.3, chan: 22, lsnr: 15.0, rssi: -85.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 4567
time: 2019-06-09 22:41:51.137000, freq: 868.3, chan: 22, lsnr: 15.0, rssi: -92.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 665
time: 2019-06-09 23:07:41.663000, freq: 868.1, chan: 21, lsnr: 15.0, rssi: -81.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 125, size: 23, FCnt: 116
time: 2019-06-09 23:08:58.644000, freq: 868.1, chan: 21, lsnr: -6.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 4367
time: 2019-06-09 23:09:04.840000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 4367
time: 2019-06-09 23:09:59.837000, freq: 868.5, chan: 23, lsnr: 1.0, rssi

In [54]:
# Count and print the number of file within the codestral_results folder
file_count = sum(1 for _ in os.listdir(result_folder_name) if os.path.isfile(os.path.join(result_folder_name, _)))
print(f"Number of files in the folder: {file_count}")

Number of files in the folder: 542


In [55]:
# Count and print the number of file within the codestral_results folder
file_count = sum(1 for _ in os.listdir(dataset_path) if os.path.isfile(os.path.join(dataset_path, _)))
print(f"Number of files in the folder: {file_count}")

Number of files in the folder: 546


# Given the same few-shot samples ask to Codestral to generate the Python code to perform the task.

In [ ]:
prompt ="""
Given the following LOG string, write a Python function to calculate and output (print and store in a file) how many frames are received per each dev_eui.

LOG:
time: 2019-05-21 18:16:47.648000, freq: 868.5, chan: 23, lsnr: 5.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4130
time: 2019-05-21 18:19:00.647000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4142
time: 2019-05-21 18:25:56.343000, freq: 868.1, chan: 21, lsnr: -2.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:02.544000, freq: 868.1, chan: 21, lsnr: -7.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:08.678000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:38:09.529000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 3958
time: 2019-05-21 18:39:57.539000, freq: 868.3, chan: 22, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4137
time: 2019-05-21 18:40:03.735000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4137
time: 2019-05-21 18:46:42.609000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -92.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 25, FCnt: 4088
time: 2019-05-21 18:48:29.241000, freq: 868.1, chan: 21, lsnr: 14.0, rssi: -90.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 186
time: 2019-05-21 19:16:46.731000, freq: 868.3, chan: 22, lsnr: 5.0, rssi: -108.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4131
time: 2019-05-21 19:19:00.174000, freq: 868.5, chan: 23, lsnr: -8.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4143
time: 2019-05-21 19:25:55.447000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:26:01.582000, freq: 868.5, chan: 23, lsnr: -5.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:26:07.785000, freq: 868.5, chan: 23, lsnr: -7.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:38:09.158000, freq: 868.5, chan: 23, lsnr: -1.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 25, FCnt: 3959
time: 2019-05-21 19:39:56.692000, freq: 868.1, chan: 21, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4138
time: 2019-05-21 19:40:02.892000, freq: 868.1, chan: 21, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4138
time: 2019-05-21 19:46:41.701000, freq: 868.5, chan: 23, lsnr: 8.0, rssi: -106.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 25, FCnt: 4089
time: 2019-05-21 19:48:28.310000, freq: 868.1, chan: 21, lsnr: 13.0, rssi: -87.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 187
time: 2019-05-21 20:12:05.544000, freq: 868.5, chan: 23, lsnr: 0.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 3889


OUTPUT:
For DEVICE dev_eui:120 all received frames are 2.
For DEVICE dev_eui:121 all received frames are 4.
For DEVICE dev_eui:122 all received frames are 2.
For DEVICE dev_eui:123 all received frames are 2.
For DEVICE dev_eui:124 all received frames are 6.
For DEVICE dev_eui:126 all received frames are 2.
For DEVICE dev_eui:127 all received frames are 2.
For DEVICE dev_eui:128 all received frames are 1.

"""

In [ ]:
message = [
    {
        "role": "user",
        "content": prompt
    }
]

chat_response = client.chat.complete(
    model=model,
    messages=message
)

result = chat_response.choices[0].message.content
print(result+"\n\n")

Here is a Python function that reads the given LOG string, processes it, and outputs the number of frames received per device. The function also writes the output to a file named `frames_per_device.txt`.

```python
import re

def process_log(log_string):
    devices = {}

    for line in log_string.split('\n'):
        match = re.search(r'dev_eui:\s*(\d+)', line)
        if match:
            dev_eui = int(match.group(1))
            fcnt = devices.get(dev_eui, 0)
            devices[dev_eui] = fcnt + 1

    for dev_eui, count in devices.items():
        print(f"For DEVICE dev_eui:{dev_eui} all received frames are {count}")

    with open('frames_per_device.txt', 'w') as f:
        for dev_eui, count in devices.items():
            f.write(f"For DEVICE dev_eui:{dev_eui} all received frames are {count}\n")

log_string = """
time: 2019-05-21 18:16:47.648000, freq: 868.5, chan: 23, lsnr: 5.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4130
time: 2019-05-21 18

In [ ]:
# Let's write a function to use this code.
# To be adherent as much as possible to the original question we try to not change almost anything.
import re

def process_log(log_string):
    devices = {}

    result = ""

    for line in log_string.split('\n'):
        match = re.search(r'dev_eui:\s*(\d+)', line)
        if match:
            dev_eui = int(match.group(1))
            fcnt = devices.get(dev_eui, 0)
            devices[dev_eui] = fcnt + 1

    devices = {key: devices[key] for key in sorted(devices)}

    for dev_eui, count in devices.items():
      # We add a '.' and a \n a the end of the string
      result += (f"For DEVICE dev_eui:{dev_eui} all received frames are {count}.\n")
      #print(f"For DEVICE dev_eui:{dev_eui} all received frames are {count}")

    return result[:-1]

log_string = """
time: 2019-05-21 18:16:47.648000, freq: 868.5, chan: 23, lsnr: 5.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4130
time: 2019-05-21 18:19:00.647000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4142
time: 2019-05-21 18:25:56.343000, freq: 868.1, chan: 21, lsnr: -2.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:02.544000, freq: 868.1, chan: 21, lsnr: -7.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:08.678000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:38:09.529000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 3958
time: 2019-05-21 18:39:57.539000, freq: 868.3, chan: 22, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4137
time: 2019-05-21 18:40:03.735000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4137
time: 2019-05-21 18:46:42.609000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -92.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 25, FCnt: 4088
time: 2019-05-21 18:48:29.241000, freq: 868.1, chan: 21, lsnr: 14.0, rssi: -90.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 186
time: 2019-05-21 19:16:46.731000, freq: 868.3, chan: 22, lsnr: 5.0, rssi: -108.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4131
time: 2019-05-21 19:19:00.174000, freq: 868.5, chan: 23, lsnr: -8.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4143
time: 2019-05-21 19:25:55.447000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:26:01.582000, freq: 868.5, chan: 23, lsnr: -5.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:26:07.785000, freq: 868.5, chan: 23, lsnr: -7.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:38:09.158000, freq: 868.5, chan: 23, lsnr: -1.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 25, FCnt: 3959
time: 2019-05-21 19:39:56.692000, freq: 868.1, chan: 21, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4138
time: 2019-05-21 19:40:02.892000, freq: 868.1, chan: 21, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4138
time: 2019-05-21 19:46:41.701000, freq: 868.5, chan: 23, lsnr: 8.0, rssi: -106.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 25, FCnt: 4089
time: 2019-05-21 19:48:28.310000, freq: 868.1, chan: 21, lsnr: 13.0, rssi: -87.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 187
time: 2019-05-21 20:12:05.544000, freq: 868.5, chan: 23, lsnr: 0.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 3889
"""

process_log(log_string)

'For DEVICE dev_eui:120 all received frames are 2.\nFor DEVICE dev_eui:121 all received frames are 4.\nFor DEVICE dev_eui:122 all received frames are 2.\nFor DEVICE dev_eui:123 all received frames are 2.\nFor DEVICE dev_eui:124 all received frames are 6.\nFor DEVICE dev_eui:126 all received frames are 2.\nFor DEVICE dev_eui:127 all received frames are 2.\nFor DEVICE dev_eui:128 all received frames are 1.'

## Let's run the code.

In [ ]:
!rm -r 'codestral_source_code_results/'

In [ ]:
# Define the directory path containing the results.
folder_name = 'codestral_source_code_results/'
#directory_path = os.path.join(os.getcwd(), folder_name)
# Create the directory if it doesn't exist
if not os.path.exists(folder_name):
  os.makedirs(folder_name)

# Iterate through all the test_files
for filename in test_set:
        file_path = folder_name+filename
        print(filename)
        print(file_path)
        if(os.path.isfile(file_path)):
          print("Already done!\n\n")
          continue
        # Check if the file is a text file
        if filename.endswith(".txt"):

            # Open the file and read its contents
            with open(os.path.join(folder_path, filename), 'r') as file:
                # Read the contents of the file and append it to all_contents
                file_contents = file.read()

                # Remove the ANSWER in the original log file.
                file_contents = file_contents.split("ANSWER", 1)

                result = process_log(file_contents[0])

                print(result+"\n\n")

                with open(file_path, 'w') as f:
                  # Write some content to the file
                  f.write(result)

Streaming output truncated to the last 5000 lines.
For DEVICE dev_eui:128 all received frames are 4.


prompt_n_hours=2_skip_hours=268.txt
mistral_7b_source_code_results/prompt_n_hours=2_skip_hours=268.txt
For DEVICE dev_eui:120 all received frames are 4.
For DEVICE dev_eui:121 all received frames are 2.
For DEVICE dev_eui:122 all received frames are 2.
For DEVICE dev_eui:123 all received frames are 2.
For DEVICE dev_eui:124 all received frames are 4.
For DEVICE dev_eui:126 all received frames are 2.
For DEVICE dev_eui:127 all received frames are 2.
For DEVICE dev_eui:128 all received frames are 4.


prompt_n_hours=2_skip_hours=924.txt
mistral_7b_source_code_results/prompt_n_hours=2_skip_hours=924.txt
For DEVICE dev_eui:120 all received frames are 2.
For DEVICE dev_eui:121 all received frames are 1.
For DEVICE dev_eui:122 all received frames are 1.
For DEVICE dev_eui:123 all received frames are 3.
For DEVICE dev_eui:124 all received frames are 1.
For DEVICE dev_eui:125 all received fra

# Here the function to run the syntactic evaluation at file-level and at global-level.

In [20]:
def evaluate_results (dataset_path,result_path):
  dataset_corpus = ""
  results_corpus = ""
  # Iterate through all the test_files
  for filename in test_set:
      # Specify the paths to the files
      file1_path = result_path+filename
      file2_path = dataset_path+filename

      # Read the contents of the files
      try:
        with open(file1_path, 'r', encoding='utf-8') as file1:
          text1 = file1.read()
      except:
        continue
      try:
        with open(file2_path, 'r', encoding='utf-8') as file2:
          text2 = file2.read()
      except:
        continue

      # Find the index where "ANSWER:" starts
      answer_start_index = text2.find("ANSWER:")

      # Remove all content before and excluding "ANSWER:"
      if answer_start_index != -1:
          text2 = text2[answer_start_index + len("ANSWER:"):].strip()

      print("Sample: ", filename)

      print(text2)
      print("\n")
      print(text1)

      dataset_corpus += text2
      results_corpus += text1

      # Tokenize the texts
      reference = text1.split()
      candidate = text2.split()

      # Compute BLEU score
      smoothing_function = SmoothingFunction().method1
      bleu_score = sentence_bleu([reference], candidate, smoothing_function=smoothing_function)

      # Compute ROUGE score
      scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
      rouge_scores = scorer.score(text1, text2)

      # Print the results
      print(f"BLEU score: {bleu_score}")
      print(f"ROUGE-1: {rouge_scores['rouge1']}")
      print(f"ROUGE-L: {rouge_scores['rougeL']}")

      print("\n\n")
  # Now let's print the results on the whole corpus.
  print("\n\nTHE RESULT ON THE TWO CORPORA ARE:\n")
  # Tokenize the texts
  reference = results_corpus.split()
  candidate = dataset_corpus.split()

  # Compute BLEU score
  smoothing_function = SmoothingFunction().method1
  bleu_score = sentence_bleu([reference], candidate, smoothing_function=smoothing_function)

  # Compute ROUGE score
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
  rouge_scores = scorer.score([reference], candidate)

  # Print the results
  print(f"BLEU score: {bleu_score}")
  print(f"ROUGE-1: {rouge_scores['rouge1']}")
  print(f"ROUGE-L: {rouge_scores['rougeL']}")

## Let's evaluate the results of Codestral.

In [56]:
dataset_path = "prompt_Q1-2024-05-20/"
result_path = "codestral_results/"
evaluate_results(dataset_path,result_path)

Streaming output truncated to the last 5000 lines.

For DEVICE dev_eui:120 all received frames are 2.
For DEVICE dev_eui:121 all received frames are 1.
For DEVICE dev_eui:122 all received frames are 2.
For DEVICE dev_eui:123 all received frames are 2.
For DEVICE dev_eui:124 all received frames are 3.
For DEVICE dev_eui:126 all received frames are 2.
For DEVICE dev_eui:127 all received frames are 2.
For DEVICE dev_eui:128 all received frames are 3.

For DEVICE dev_eui:120 all received frames are 2.
For DEVICE dev_eui:122 all received frames are 2.
For DEVICE dev_eui:124 all received frames are 3.
For DEVICE dev_eui:128 all received frames are 3.
BLEU score: 0.5696730216492468
ROUGE-1: Score(precision=0.975, recall=0.65, fmeasure=0.78)
ROUGE-L: Score(precision=0.95, recall=0.6333333333333333, fmeasure=0.7599999999999999)



Sample:  prompt_n_hours=2_skip_hours=698.txt
For DEVICE dev_eui:120 all received frames are 2.
For DEVICE dev_eui:121 all received frames are 2.
For DEVICE dev_eui:12

AttributeError: 'list' object has no attribute 'lower'

## Let's evaluate the results of the source code generated by Codestral.

In [ ]:
dataset_path = "prompt_Q1-2024-05-20/"
result_path = "codestral_source_code_results/"
evaluate_results(dataset_path,result_path)

Streaming output truncated to the last 5000 lines.
For DEVICE dev_eui:120 all received frames are 2.
For DEVICE dev_eui:121 all received frames are 2.
For DEVICE dev_eui:122 all received frames are 1.
For DEVICE dev_eui:123 all received frames are 3.
For DEVICE dev_eui:124 all received frames are 2.
For DEVICE dev_eui:126 all received frames are 2.
For DEVICE dev_eui:127 all received frames are 2.
For DEVICE dev_eui:128 all received frames are 4.
BLEU score: 1.0
ROUGE-1: Score(precision=1.0, recall=1.0, fmeasure=1.0)
ROUGE-L: Score(precision=1.0, recall=1.0, fmeasure=1.0)



Sample:  prompt_n_hours=2_skip_hours=362.txt
For DEVICE dev_eui:120 all received frames are 2.
For DEVICE dev_eui:121 all received frames are 4.
For DEVICE dev_eui:122 all received frames are 2.
For DEVICE dev_eui:123 all received frames are 2.
For DEVICE dev_eui:124 all received frames are 4.
For DEVICE dev_eui:126 all received frames are 2.
For DEVICE dev_eui:127 all received frames are 2.
For DEVICE dev_eui:128 

#Let's evaluate the correctness of the responses in terms of Precision and Recall.

In [32]:
import re

def calculate_precision_recall (dataset_path,result_path):
  total_true_positive = 0
  total_false_positive = 0
  total_false_negative = 0
  # Iterate through all the test_files
  for filename in test_set:
      # Specify the paths to the files
      file1_path = result_path+filename
      file2_path = dataset_path+filename

      # Read the contents of the files
      try:
        with open(file1_path, 'r', encoding='utf-8') as file1:
          text1 = file1.read()
      except:
        continue
      try:
        with open(file2_path, 'r', encoding='utf-8') as file2:
          text2 = file2.read()
      except:
        continue

      # Find the index where "ANSWER:" starts
      answer_start_index = text2.find("ANSWER:")

      # Remove all content before and excluding "ANSWER:"
      if answer_start_index != -1:
          text2 = text2[answer_start_index + len("ANSWER:"):].strip()

      print("Sample: ", filename)

      #print(text2)
      #print("\n")
      #print(text1)

      # text1 contains results. text2 contains the correct values.
      # Create a dictionary for both text files. Key is the dev_eui and value is the number of frames.

      # For each line in text1 create a pair key value. key is the number of dev_eui, and value is the number of frames.
      results_dict = {}
      target_dict = {}

      #print(text1)

      # Split the data into lines
      lines = text1.split('\n')

      #print(lines)

      for line in lines:
        parts = line.split()
        try:
          dev_eui = parts[2].split(':')[1]
          frames = parts[7].split('.')[0]
        except:
          continue

        # Add to dictionary
        try:
          results_dict[dev_eui] = int(frames)
        except:
          results_dict[dev_eui] = 0

      # Split the data into lines
      lines = text2.split('\n')

      #print(lines)

      for line in lines:
        #print(line)
        # Split the line to extract dev_eui and frames
        parts = line.split()
        try:
          dev_eui = parts[2].split(':')[1]
          frames = parts[7].split('.')[0]
        except:
          continue

        # Add to dictionary
        try:
          target_dict[dev_eui] = int(frames)
        except:
          target_dict[dev_eui] = 0

      print(results_dict)
      print(target_dict)

      # Let's count the true positive.
      true_positive = 0
      for key in results_dict:
        if key in target_dict:
          if results_dict[key] == target_dict[key]:
            true_positive += 1
      print("True positive so far: ",true_positive)


      # Let's count the false positive.
      false_positive = 0
      for key in results_dict:
        if key not in target_dict:
          false_positive += 1
        else:
          if key in target_dict and results_dict[key] != target_dict[key]:
            false_positive += 1
      print("False positive so far: ",false_positive)

      # Let's count the false negative.
      false_negative = 0
      for key in target_dict:
        if key not in results_dict:
          false_negative += 1
      print("False negative so far: ",false_negative)

      total_true_positive += true_positive
      total_false_positive += false_positive
      total_false_negative += false_negative

  print("The precision is: ",total_true_positive/(total_true_positive+total_false_positive))
  print("The recall is: ",total_true_positive/(total_true_positive+total_false_negative))

##Let's evaluate Codestral

In [58]:
dataset_path = "prompt_Q1-2024-05-20/"
result_path = "codestral_results/"
calculate_precision_recall(dataset_path,result_path)

Sample:  prompt_n_hours=2_skip_hours=248.txt
{'11': 0, '19': 0, '31': 0, '48': 0, '49': 0, '58': 0, '00': 0, '02': 0}
{'120': 1, '121': 2, '123': 3, '127': 1, '128': 2}
True positive so far:  0
False positive so far:  8
False negative so far:  5
Sample:  prompt_n_hours=2_skip_hours=786.txt
{'120': 2, '121': 2, '122': 3, '123': 2, '124': 3, '125': 2, '126': 2, '127': 2, '128': 6, '129': 4}
{'120': 2, '121': 2, '122': 2, '123': 2, '124': 4, '125': 2, '126': 2, '127': 2, '128': 5, '129': 4}
True positive so far:  7
False positive so far:  3
False negative so far:  0
Sample:  prompt_n_hours=2_skip_hours=1078.txt
{'120': 2, '121': 1, '122': 2, '123': 1, '124': 3, '125': 2, '126': 2, '127': 2, '128': 1, '129': 2}
{'120': 2, '121': 2, '122': 2, '123': 1, '124': 4, '125': 2, '126': 2, '127': 2, '128': 2, '129': 2}
True positive so far:  7
False positive so far:  3
False negative so far:  0
Sample:  prompt_n_hours=2_skip_hours=686.txt
{'120': 2, '121': 2, '122': 2, '123': 2, '124': 4, '125': 2,

## Let's evaluate Codestral with CoC (Chain of Code)

In [ ]:
dataset_path = "prompt_Q1-2024-05-20/"
result_path = "codestral_source_code_results/"
calculate_precision_recall(dataset_path,result_path)

# Zip and download the results using the two approaches.

In [57]:
!zip -r codestral_results.zip codestral_results
!zip -r codestral_source_code_results.zip codestral_source_code_results

updating: codestral_results/ (stored 0%)
updating: codestral_results/prompt_n_hours=2_skip_hours=74.txt (deflated 79%)
updating: codestral_results/prompt_n_hours=2_skip_hours=100.txt (deflated 77%)
updating: codestral_results/prompt_n_hours=2_skip_hours=36.txt (deflated 78%)
updating: codestral_results/prompt_n_hours=2_skip_hours=760.txt (deflated 82%)
updating: codestral_results/prompt_n_hours=2_skip_hours=578.txt (deflated 79%)
updating: codestral_results/prompt_n_hours=2_skip_hours=412.txt (deflated 75%)
updating: codestral_results/prompt_n_hours=2_skip_hours=456.txt (deflated 79%)
updating: codestral_results/prompt_n_hours=2_skip_hours=972.txt (deflated 81%)
updating: codestral_results/prompt_n_hours=2_skip_hours=438.txt (deflated 81%)
updating: codestral_results/prompt_n_hours=2_skip_hours=586.txt (deflated 80%)
updating: codestral_results/prompt_n_hours=2_skip_hours=4.txt (deflated 79%)
updating: codestral_results/prompt_n_hours=2_skip_hours=294.txt (deflated 77%)
updating: codes